In [22]:
import serial
import time
import re

In [23]:
port = "COM8"
baud_rate = 128000
data_bits = 8
stop_bits = 1
parity = serial.PARITY_NONE

lidar_command = {
    "start_scan":"A5 60", 
    "stop_scan":"A5 65", 
    "get_info":"A5 90", 
    "get_health":"A5 91", 
    "soft_restart":"A5 80", 
}

hex_re = re.compile(r'\s+')

In [152]:
class LidarResponse:
    def __init__(self, response):
        self.response_raw = response
        self.response_list = self.split_response(self.response_raw)
        self.n_response = len(self.response_list)
                
    def split_response(self, data):
        content_groups = re.findall(r'(?s)AA55(.*?)AA55', data)
        complete_groups = [group for group in content_groups if len(group) % 2 == 0]
        complete_groups_with_delimiter = ["AA55" + group for group in complete_groups]
        return complete_groups_with_delimiter

    def convert_l_endian(self, data):
        bytes_list = [data[i:i+2] for i in range(0, len(data), 2)]
        little_endian_list = [bytes_list[i+1] + bytes_list[i] for i in range(0, len(bytes_list), 2)]
        result_str = ''.join(little_endian_list)
        return result_str

In [147]:
def send_command(serial_object: serial.Serial, command: str):
    if command in lidar_command:
        command_hex = lidar_command[command]
        data_bytes = bytes.fromhex(hex_re.sub('', command_hex))
        serial_object.write(data_bytes)
        response = serial_object.read_until().hex().upper()
        return response
    else:
        return None

In [62]:
ser = serial.Serial(
    port, 
    baud_rate, 
    bytesize=data_bits, 
    parity=parity, 
    stopbits=stop_bits, 
    timeout=1
    )

In [150]:
one_scan = send_command(ser, "start_scan")
one_scan

'00000000AA55000C8BAACBAD724600000000000000000000541A081AD819A81968190819D418AA5501011BAE1BAE7F4CD418AA55002869AEDF0510CEE418941884187C189018000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006C0370036C03680368036403AA55002825068F116A6860035D0361035903550355035103510351034D03490349034503450345034503410341033D033D0339033903390339033903390339033503350339030000760200002D032D0300000000000030033503AA550028DD11491D57723503390339033D033D033D034103410341034103410345034503490349034D034D034D035103550355035903590359036103610364036503680368036C03700374037403740378030000740300000000AA550028971D0D29344900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000190421042904000048063C0600006104AA5500285B29D134816E61045D045804000000000000F503000000000000000000000000D4169416000000005C174C17000000008018D81820190000EC0F0000CC0F880F940F1C10181028100000000000004C0644060000000

In [87]:
def split_response(response):
    content_groups = re.findall(r'(?s)AA55(.*?)AA55', response)
    complete_groups = [group for group in content_groups if len(group) % 2 == 0]
    complete_groups_with_delimiter = ["AA55" + group for group in complete_groups]
    return complete_groups_with_delimiter

In [121]:
def bytes_to_little_endian(input_str):
    bytes_list = [input_str[i:i+2] for i in range(0, len(input_str), 2)]
    little_endian_list = [bytes_list[i+1] + bytes_list[i] for i in range(0, len(bytes_list), 2)]
    result_str = ''.join(little_endian_list)
    return result_str

In [108]:
def print_bytes(data):
    for i in range(0, len(data), 2):
        byte = data[i:i+2]
        print(f"Byte {i//2}: {byte}")

In [151]:
response = split_response(one_scan)
response

['AA55000C8BAACBAD724600000000000000000000541A081AD819A81968190819D418',
 'AA55002869AEDF0510CEE418941884187C189018000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006C0370036C03680368036403',
 'AA550028DD11491D57723503390339033D033D033D034103410341034103410345034503490349034D034D034D035103550355035903590359036103610364036503680368036C03700374037403740378030000740300000000',
 'AA5500285B29D134816E61045D045804000000000000F503000000000000000000000000D4169416000000005C174C17000000008018D81820190000EC0F0000CC0F880F940F1C10181028100000000000004C06440600000000']

In [126]:
print_bytes(one_resp)

Byte 0: 55
Byte 1: AA
Byte 2: 28
Byte 3: 00
Byte 4: 4C
Byte 5: 81
Byte 6: 58
Byte 7: 05
Byte 8: 6F
Byte 9: 5B
Byte 10: 00
Byte 11: 00
Byte 12: 00
Byte 13: 00
Byte 14: 05
Byte 15: A4
Byte 16: 05
Byte 17: 90
Byte 18: 05
Byte 19: 7C
Byte 20: 05
Byte 21: 69
Byte 22: 05
Byte 23: 55
Byte 24: 05
Byte 25: 41
Byte 26: 05
Byte 27: 31
Byte 28: 05
Byte 29: 1D
Byte 30: 05
Byte 31: 0D
Byte 32: 04
Byte 33: FD
Byte 34: 04
Byte 35: ED
Byte 36: 04
Byte 37: DD
Byte 38: 04
Byte 39: CD
Byte 40: 04
Byte 41: BD
Byte 42: 04
Byte 43: B1
Byte 44: 04
Byte 45: A1
Byte 46: 04
Byte 47: 95
Byte 48: 04
Byte 49: 89
Byte 50: 04
Byte 51: 79
Byte 52: 04
Byte 53: 6D
Byte 54: 04
Byte 55: 65
Byte 56: 04
Byte 57: 59
Byte 58: 04
Byte 59: 4D
Byte 60: 04
Byte 61: 41
Byte 62: 04
Byte 63: 39
Byte 64: 04
Byte 65: 2D
Byte 66: 04
Byte 67: 25
Byte 68: 04
Byte 69: 1D
Byte 70: 04
Byte 71: 11
Byte 72: 04
Byte 73: 09
Byte 74: 04
Byte 75: 01
Byte 76: 03
Byte 77: F9
Byte 78: 03
Byte 79: F1
Byte 80: 03
Byte 81: E9
Byte 82: 03
Byte 83: E1
By

In [133]:
def parse_hex_string(hex_str):
    result_dict = {}
    mappings = {'fsa': 8, 'lsa': 12, 'cs': 16}

    for key, start_index in mappings.items():
        end_index = start_index + 4
        result_dict[key] = hex_str[start_index:end_index]

    i = 1
    while (start_index := 20 + (i - 1) * 4) < len(hex_str):
        end_index = start_index + 4
        result_dict[f's{i}'] = hex_str[start_index:end_index]
        i += 1

    return result_dict

In [138]:
parsed_response = parse_hex_string(one_resp)
fsa = parsed_response["fsa"]
lsa = parsed_response["lsa"]

In [143]:
def get_angle(hex_angle):
    return (int(hex_angle, 16) >> 1)/ 64

In [145]:
get_angle(fsa), get_angle(lsa)

(153.0, 176.03125)

In [153]:
ser.close()